In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
from plotly.subplots import make_subplots

In [2]:
# Read noisy heart rate data
df = pl.read_csv("noisy_heart_rate_time_series.csv")
time = df["t"].to_numpy()
hr_accurate_orig = df["heart_rate_accurate"].to_numpy()
hr_noisy_orig = df["heart_rate_noisy"].to_numpy()


In [3]:
# Normalize data
hr_accurate_mean = np.mean(hr_accurate_orig)
hr_accurate_std = np.std(hr_accurate_orig)
hr_accurate = (hr_accurate_orig - hr_accurate_mean) / hr_accurate_std
hr_noisy = (hr_noisy_orig - hr_accurate_mean) / hr_accurate_std

In [4]:
# Plot original data and normalized data using plotly
fig = make_subplots(rows=2, cols=1, subplot_titles=("Original data", "Normalized data"))

fig.add_trace(
    go.Scatter(x=time, y=hr_accurate_orig, mode="lines", name="Accurate"), row=1, col=1
)
fig.add_trace(
    go.Scatter(x=time, y=hr_noisy_orig, mode="lines", name="Noisy"), row=1, col=1
)

fig.add_trace(
    go.Scatter(x=time, y=hr_accurate, mode="lines", name="Accurate"), row=2, col=1
)
fig.add_trace(go.Scatter(x=time, y=hr_noisy, mode="lines", name="Noisy"), row=2, col=1)

fig.update_yaxes(title_text="Heart rate", row=1, col=1)
fig.update_yaxes(title_text="Normalized heart rate", row=2, col=1)
fig.update_layout(height=600, width=800, showlegend=True)
fig.show()

In [18]:
w1 = 0.1
w2 = 0.1
b = 0.0
learning_rate = 0.01  # 0.005
# learning_rate = 0.1
# learning_rate_w1 = 0.00001
# learning_rate_w2 = 0.00001
# learning_rate_b = 0.1

n_epochs = 20

In [19]:
weights_history = [[w1, w2, b]]
for _ in range(n_epochs):
    hr_pred = []
    y_prev = 0.0
    for x, y in zip(hr_noisy, hr_accurate):
        y_pred = w1 * x + w2 * y_prev + b
        hr_pred.append(y_pred)
        error = y_pred - y
        update = -learning_rate * 2 * error  # Neg sign to do gradient descent
        w1 += update * x
        w2 += update * y_prev
        b += update

        y_prev = y_pred
    # print(f"w1: {w1}, w2: {w2}, b: {b}")
    weights_history.append([w1, w2, b])

weights_history = np.array(weights_history)
# history_df = pl.DataFrame(weights_history, schema=["w1", "w2", "b"], orient="row")


In [20]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=weights_history[:, 0],
        mode="lines",
        name="w1",
    )
)
fig.add_trace(
    go.Scatter(
        y=weights_history[:, 1],
        mode="lines",
        name="w2",
    )
)
fig.add_trace(
    go.Scatter(
        y=weights_history[:, 2],
        mode="lines",
        name="b",
    )
)

fig.update_layout(
    title="Weights over epochs",
    xaxis_title="Epochs",
    yaxis_title="Weight value",
)
fig.show()

In [21]:
# Plot predictions using plotly


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y=hr_noisy * hr_accurate_std + hr_accurate_mean,
        mode="lines",
        name="Input",
        opacity=0.3,
    )
)
fig.add_trace(
    go.Scatter(
        y=np.array(hr_pred) * hr_accurate_std + hr_accurate_mean,
        mode="lines",
        name="Predicted",
    )
)
fig.add_trace(
    go.Scatter(
        y=hr_accurate * hr_accurate_std + hr_accurate_mean,
        mode="lines",
        name="Accurate",
    )
)
fig.update_layout(
    title="RNN Prediction vs Input and Accurate",
    xaxis_title="Time",
    yaxis_title="Heart Rate",
)
fig.show()